In [1]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from scipy.linalg import sqrtm
import tensorflow as tf
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from make_dataset import process_image, parse_and_save_dir

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
def load_dataset(image_dir):
    data_list = []
    fn_list = [x for x in os.listdir(image_dir) if ".tif" in x]
    for fn in fn_list:
        input_file = image_dir + fn
        data = process_image(input_file)
        data_list.append(data)
    return data_list

# calculate frechet inception distance
def calculate_fid(model, images1, images2):
    # calculate activations
    act1 = model.predict(images1)
    act2 = model.predict(images2)
    
    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)

    # calculate sum squared difference between means
    ssdiff  = np.sum((mu1 - mu2)**2.0)
    # calculate sqrt of product between cov
    covmean  = sqrtm(sigma1.dot(sigma2))

    # check and correct imaginary numbers from sqrt
    if iscomplexobj(covmean):
        covmean = covmean.real
        
    # calculate score
    fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)

    return fid

def expand_channels(data_list):
    lx, ly, _ = data_list[0].shape
    new_data = []
    for data in data_list:
        expanded_data = np.zeros((lx, ly, 3))
        expanded_data[:,:,0] = data[:,:,0]
        data_fft = np.fft.fftshift(np.fft.fft2(data[:,:,0]))
        rfft = data_fft.real 
        ifft = data_fft.imag
        power = rfft*rfft + ifft*ifft
        expanded_data[:,:,1] = np.log10(power)
        new_data.append(expanded_data)
    return np.array(new_data)

In [3]:
image_gan_dir = "/mnt/d/GANNED_SIM_20230111_MODEL_unet_ident_gen_fft_bincross_10_SIM_pristine_gaussian_0.1_EXP_Gr_3/"
image_exp_dir = "/mnt/d/stem-learning/data/Gr/GR_3/exp_1K_256_slices/"

image_gan = preprocess_input(expand_channels(load_dataset(image_gan_dir)))
image_exp = preprocess_input(expand_channels(load_dataset(image_exp_dir)))

model = InceptionV3(include_top=False, pooling='avg', input_shape=(256,256,3))
fid = calculate_fid(model, image_exp, image_gan)
print('          FID (exp, gan): %.3f' % fid)


35/35 [==============================] - 1s 33ms/step
          FID (exp, gan): 0.477


In [ ]:
for idx in [1, 2, 3]:
    print("IDX = {}".format(idx))
    image_dir1 = "D:/Gr/Gr_{}/exp_1K_256_slices/".format(idx)
    images1 = expand_channels(load_dataset(image_dir1))
    images1 = preprocess_input(images1)

    for i in [0.1]:
        print("gaussian = {}".format(i))
        image_dir2 = "D:/Gr/Gr_{}/sim_1K_gauss_{}_256_slices/".format(idx, i)
        image_dir3 = "D:/Gr/Gr_{}/GANNED_SIM/exp/sim_abberation_20220727_MODEL_unet_ident_gen_fft_10_SIM_pristine_gaussian_{}_EXP_Gr_{}/".format(idx, i, idx)

        images2 = expand_channels(load_dataset(image_dir2))
        images3 = expand_channels(load_dataset(image_dir3))

        images2 = preprocess_input(images2)
        images3 = preprocess_input(images3)
        
        model = InceptionV3(include_top=False, pooling='avg', input_shape=(256,256,3))
        fid1, fid2 = calculate_fid(model, images1, images2, images3)
        print('FID (exp, sim): %.3f' % fid1)
        print('FID (exp, gan): %.3f' % fid2)

IDX = 1
gaussian = 0.1
35/35 [==============================] - 1s 25ms/step
FID (exp, sim): 24.002
FID (exp, gan): 8.749
IDX = 2
gaussian = 0.1
35/35 [==============================] - 1s 26ms/step
FID (exp, sim): 26.843
FID (exp, gan): 3.763
IDX = 3
gaussian = 0.1
35/35 [==============================] - 1s 27ms/step
FID (exp, sim): 32.690
FID (exp, gan): 1.661


: 

In [3]:
for idx in [3]:
    print("IDX = {}".format(idx))
    image_dir1 = "D:/STO/STO_{}/exp_1K_256_slices/".format(idx)
    images1 = expand_channels(load_dataset(image_dir1))
    images1 = preprocess_input(images1)[:1000]

    for i in [0.1]:
        print("     gaussian = {}".format(i))
        image_dir2 = "D:/STO/STO_{}/sim_1K_gauss_{}_256_slices/".format(idx, i)
        image_dir3 = "D:/STO/STO_{}/GANNED_SIM/exp/sim_abberation_20220727_MODEL_unet_ident_gen_fft_10_SIM_pristine_gaussian_{}_EXP_STO_{}/".format(idx, i, idx)

        images2 = expand_channels(load_dataset(image_dir2))
        images3 = expand_channels(load_dataset(image_dir3))

        images2 = preprocess_input(images2)[:1000]
        images3 = preprocess_input(images3)[:1000]
        
        model = InceptionV3(include_top=False, pooling='avg', input_shape=(256,256,3))
        fid1, fid2 = calculate_fid(model, images1, images2, images3)
        print('          FID (exp, sim): %.3f' % fid1)
        print('          FID (exp, gan): %.3f' % fid2)

IDX = 3
     gaussian = 0.1
32/32 [==============================] - 1s 25ms/step
          FID (exp, sim): 19.595
          FID (exp, gan): 0.474


In [8]:
ganned_sim_dir = "/mnt/d/stem-learning/data/WSe/data_for_gan/generated/GANNED_SIM/exp/"

for idfn in ["107", "212", "all"]:
    identifier = "sim_abberation_20220705_MODEL_unet_ident_gen_fft_10_SIM_pristine_gaussian_0.1_EXP_{}".format(idfn)
    image_dir = "{}{}/".format(ganned_sim_dir, identifier)
    parse_and_save_dir(image_dir, save_dir=image_dir[:-1]+"_256_slices/")

In [15]:
image_exp_dir = "D:/stem-learning/data/WSe/data_for_gan/experiment/107_normalized_256_slices/"
image_man_dir = "D:/data_for_chiahao/WSe/manual_sim_256_slices/"
image_gan_dir = "D:/data_for_chiahao/WSe/GAN generated_256_slices/"
image_sim_dir = "D:/data_for_chiahao/WSe/sim_pristine_256_slices/"

image_exp = expand_channels(load_dataset(image_exp_dir))
image_man = expand_channels(load_dataset(image_man_dir))
image_gan = expand_channels(load_dataset(image_gan_dir))
image_sim = expand_channels(load_dataset(image_sim_dir))

image_exp = preprocess_input(image_exp)
image_man = preprocess_input(image_man)
image_gan = preprocess_input(image_gan)
image_sim = preprocess_input(image_sim)

model = InceptionV3(include_top=False, pooling='avg', input_shape=(256,256,3))
fid1, fid2 = calculate_fid(model, image_exp, image_man, image_gan)
fid3, fid4 = calculate_fid(model, image_exp, image_man, image_sim)
print('          FID (exp, man): %.3f' % fid1)
print('          FID (exp, gan): %.3f' % fid2)
print('          FID (exp, sim): %.3f' % fid4)

54/54 [==============================] - 2s 29ms/step
          FID (exp, man): 0.727
          FID (exp, gan): 0.354
          FID (exp, sim): 31.864


In [44]:
for dataset in ["107_1840", "107_1847", "211_1528", "211_1922", "211_2124"]:
    print(dataset)
    identifier = "20220705_MODEL_unet_ident_gen_fft_10_SIM_pristine_gaussian_0.1_EXP_{}".format(dataset)
    image_gan_dir_all = "D:/stem-learning/data/WSe/data_for_gan/generated/GANNED_SIM/exp/sim_abberation_{}/".format(identifier)
    image_gan_dir = "D:/stem-learning/data/WSe/data_for_gan/generated/GANNED_256/exp/sim_abberation_{}_256_slices/".format(identifier)
    parse_and_save_dir(image_gan_dir_all, save_dir=image_gan_dir)

    image_exp_dir = "D:/stem-learning/data/WSe/data_for_gan/experiment/mini_experiment_batch/RR_{}_256_slices/".format(dataset)
    #image_man_dir = "D:/data_for_chiahao/WSe/manual_sim_256_slices/"
    #image_sim_dir = "D:/data_for_chiahao/WSe/sim_pristine_256_slices/"

    image_exp = preprocess_input(expand_channels(load_dataset(image_exp_dir)[:1000]))
    #image_man = preprocess_input(expand_channels(load_dataset(image_man_dir)))
    image_gan = preprocess_input(expand_channels(load_dataset(image_gan_dir)[:1000]))
    #image_sim = preprocess_input(expand_channels(load_dataset(image_sim_dir)))

    model = InceptionV3(include_top=False, pooling='avg', input_shape=(256,256,3))
    fid1, fid2 = calculate_fid(model, image_exp, image_man, image_gan)
    fid3, fid4 = calculate_fid(model, image_exp, image_man, image_sim)
    print('          FID (exp, sim): %.3f' % fid4)
    print('          FID (exp, man): %.3f' % fid1)
    print('          FID (exp, gan): %.3f' % fid2)


107_1840
54/54 [==============================] - 2s 30ms/step
          FID (exp, sim): 31.501
          FID (exp, man): 1.575
          FID (exp, gan): 0.390
107_1847
54/54 [==============================] - 2s 27ms/step
          FID (exp, sim): 31.113
          FID (exp, man): 1.333
          FID (exp, gan): 0.458
211_1528
54/54 [==============================] - 2s 27ms/step
          FID (exp, sim): 31.111
          FID (exp, man): 0.933
          FID (exp, gan): 1.135
211_1922
54/54 [==============================] - 1s 26ms/step
          FID (exp, sim): 31.620
          FID (exp, man): 0.927
          FID (exp, gan): 0.864
211_2124
54/54 [==============================] - 1s 26ms/step
          FID (exp, sim): 30.898
          FID (exp, man): 1.002
          FID (exp, gan): 1.217
